# StatsAsma - um programa de exibição de informações sobre óbitos por asma por UF e Região

Aqui fazemos a análise das informações de óbito cuja causa primária foi Asma em cada UF/ Região, para os anos de 2016 e 2017

In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import time
import os
import re
import fileinput
import math    
from math import pi
import plotly
import plotly.graph_objs as go
import plotly.plotly as py
from ipywidgets import widgets
from datetime import date
import cufflinks as cf
import plotly.offline
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
cf.go_offline()
cf.set_config_file(offline=True, world_readable=True)
init_notebook_mode(connected=True)

Nesta seção fazemos a importação dos dados CSV coletados no sistema de obitos do DATASUS, dos anos de 2016 e 2017, em http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1&acao=26&pad=31655

In [75]:
# datasets analisados - 2016 e 2017
dataSet2016 = pd.read_csv("csv/OB_DATA_2016.csv",sep=",",low_memory=False)
dataSet2017 = pd.read_csv("csv/OB_DATA_2017.csv",sep=",",low_memory=False)

# informaçoes de cidades
cidadesBR = pd.read_csv("csv/muniBR-ALL2.csv",delimiter=',')

Aqui listamos os obitos por local de domicilio filtrado pelo CID-10. **J45** para asma, **J** para todas as doenças respiratórias. O dicionário de variáveis pode ser acessado [aqui]( http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1&acao=26).

In [76]:
# Verificacao de obitos causados por asma como causa base
onlyAsma2016 = dataSet2016[dataSet2016['CAUSABAS'].str.contains('J45')]#CID ASMA
onlyAsma2017 = dataSet2017[dataSet2017['CAUSABAS'].str.contains('J45')]#CID ASMA

# Verificacao de ASMA como doença listada nas causas
alsoAsma2016 = dataSet2016[dataSet2016['LINHAA'].str.contains('J45')| dataSet2016['LINHAB'].str.contains('J45')| dataSet2016['LINHAC'].str.contains('J45')| dataSet2016['LINHAD'].str.contains('J45') | dataSet2016['LINHAII'].str.contains('J45')]
alsoAsma2017 = dataSet2017[dataSet2017['LINHAA'].str.contains('J45')| dataSet2017['LINHAB'].str.contains('J45')| dataSet2017['LINHAC'].str.contains('J45')| dataSet2017['LINHAD'].str.contains('J45') | dataSet2017['LINHAII'].str.contains('J45')]

# Verificacao de obitos causados por doenças respiratórias
allDisease2016 = dataSet2016[dataSet2016['CAUSABAS'].str.contains('J')]#CID ASMA
allDisease2017 = dataSet2017[dataSet2017['CAUSABAS'].str.contains('J')]#CID ASMA

# Dicionario de variaveis em http://tabnet.datasus.gov.br/cgi/sih/nrdescr.htm

Consolidamos os dados de 2016 e 2017 em um só ` dataFrame`

In [121]:
# Concatena os datasets de 2016 e 2017
onlyAsma = pd.concat([onlyAsma2016,onlyAsma2017])
alsoAsma = pd.concat([alsoAsma2016,alsoAsma2017])
allDisease = pd.concat([allDisease2016,allDisease2017])

Fazemos o ajuste da idade seguindo o código do dicionário de variáveis a seguir:
Idade, composto de dois subcampos. O primeiro, de 1 dígito, indica a unidade da idade, conforme a tabela a
seguir. O segundo, de dois dígitos, indica a quantidade de unidades:
- 0: Idade ignorada, o segundo subcampo e
- 1: Horas, o segundo subcampo varia de 01 a 23
- 2: Dias, o segundo subcampo varia de 01 a 29
- 3: Meses, o segundo subcampo varia de 01 a 11
- 4: Anos, o segundo subcampo varia de 00 a 99
- 5: Anos (mais de 100 anos), o segundo subcampo varia de 0 a 99, 
- exemplos:
> - 000: Idade ignorada
>- 020: 20 minutos
>- 103: 3 horas
>- 204: 4 dias
>- 305: 5 meses
>- 400: menor de 1 ano, mas não se sabe o numero de horas, dias ou meses
>- 410: 10 anos
>- 505: 105 anos

In [122]:
# Ajuste de idade para anos
onlyAsma['AGE'] = onlyAsma['IDADE'] - 400
onlyAsma[onlyAsma['AGE'] < 0]=0

alsoAsma['AGE'] = alsoAsma['IDADE'] - 400
alsoAsma[alsoAsma['AGE'] < 0]=0

allDisease['AGE'] = allDisease['IDADE'] - 400
allDisease[allDisease['AGE'] < 0]=0

In [ ]:
def int2date(argdate: int) -> date:
    year = int(argdate % 10000)
    month = int(argdate % 1000000 / 10000)
    day = int(argdate / 1000000)
    return date(year, month, day)

In [123]:
# Mes de Óbito
onlyAsma['MESOBITO'] = (onlyAsma['DTOBITO']% 1000000/ 10000).astype(int)
alsoAsma['MESOBITO'] = (alsoAsma['DTOBITO']% 1000000/ 10000).astype(int)
allDisease['MESOBITO'] = (allDisease['DTOBITO']% 1000000/ 10000).astype(int)

Aqui, transformaremos os códigos de cada município para UF e Região. A tabela de códigos é descrita a seguir:
https://ww2.ibge.gov.br/home/geociencias/geografia/redes_fluxos/gestao_do_territorio_2014/base.shtm
> Região Norte
> - 11-Rondônia/RO
> - 12-Acre/AC
> - 13-Amazonas/AM
> - 14-Roraima/RR
> - 15-Pará/PA
> - 16-Amapá/AP
> - 17-Tocantins/TO
> 
> Região Nordeste
> - 21-Maranhão/MA
> - 22-Piauí/PI
> - 23-Ceará/CE
> - 24-Rio Grande do Norte/RN
> - 25-Paraíba/PB
> - 26-Pernambuco/PE
> - 27-Alagoas/AL
> - 28-Sergipe/SE
> - 29-Bahia/BA
> 
> Região Sudeste
> - 31-Minas Gerais/MG
> - 32-Espírito Santo/ES
> - 33-Rio de Janeiro/RJ
> - 35-São Paulo/SP
> 
> Região Sul
> - 41-Paraná/PR
> - 42-Santa Catarina/SC
> - 43-Rio Grande do Sul/RS
> 
> Região Centro-Oeste
> - 50-Mato Grosso do Sul/MS
> - 51-Mato Grosso/MT
> - 52-Goiás/GO
> - 53-Distrito Federal/DF

In [124]:
# UFs por cidade
onlyAsma['CODUF'] = (onlyAsma['CODMUNRES']/10000).astype(int)
alsoAsma['CODUF'] = (alsoAsma['CODMUNRES']/10000).astype(int)
allDisease['CODUF'] = (allDisease['CODMUNRES']/10000).astype(int)

In [125]:
# regioes por estado
onlyAsma['CODREG'] = np.floor(round(onlyAsma['CODUF']/10,1)).astype(int)
alsoAsma['CODREG'] = np.floor(round(alsoAsma['CODUF']/10,1)).astype(int)
allDisease['CODREG'] = np.floor(round(allDisease['CODUF']/10,1)).astype(int)

Definir o código das regiões com base nas UFs

In [126]:
cidadesBR['CODREG']=np.floor(round(cidadesBR['CodUF']/10,1)).astype(int)

In [127]:
onlyAsma.head()

,TIPOBITO,DTOBITO,HORAOBITO,NUMSUS,NATURAL,CODMUNNATU,DTNASC,IDADE,SEXO,RACACOR,...,LINHAA,LINHAB,LINHAC,LINHAD,LINHAII,CAUSABAS,AGE,MESOBITO,CODUF,CODREG
3796,2,1012016,1200.0,NaN,822.0,221100.0,5061950.0,465,2,1.0,...,*A419,*J969,*J189,*J459,*C56X*J980,J459,65,1,53,5
4253,2,16012016,1100.0,NaN,821.0,211130.0,7041928.0,487,2,1.0,...,*A419,*J960,*J459,NaN,NaN,J459,87,1,53,5
4383,2,20012016,2215.0,NaN,835.0,351620.0,27011953.0,462,2,4.0,...,*R578,*A419,*J189,NaN,*N179*J459*I10X*F171,J459,62,1,53,5
4654,2,27012016,900.0,898002232677216.0,824.0,240800.0,22031939.0,476,2,1.0,...,*J960,*A419,*J189,*J459,*M609*E149,J459,76,1,53,5
4687,2,28012016,220.0,NaN,190.0,NaN,11011918.0,498,2,1.0,...,*A419,*J159,*J459,NaN,*D649,J459,98,1,53,5


In [128]:
alsoAsma.head()

,TIPOBITO,DTOBITO,HORAOBITO,NUMSUS,NATURAL,CODMUNNATU,DTNASC,IDADE,SEXO,RACACOR,...,LINHAA,LINHAB,LINHAC,LINHAD,LINHAII,CAUSABAS,AGE,MESOBITO,CODUF,CODREG
709,2,13032016,2140.0,NaN,812.0,120050.0,14101923.0,492,2,4.0,...,*J459,*J189,*A310,NaN,NaN,A310,92,3,12,1
2863,2,8102016,30.0,706203538898162.0,812.0,120050.0,7011967.0,449,2,4.0,...,*I469,*J459,*I10X,*J449,NaN,J448,49,10,12,1
3796,2,1012016,1200.0,NaN,822.0,221100.0,5061950.0,465,2,1.0,...,*A419,*J969,*J189,*J459,*C56X*J980,J459,65,1,53,5
4253,2,16012016,1100.0,NaN,821.0,211130.0,7041928.0,487,2,1.0,...,*A419,*J960,*J459,NaN,NaN,J459,87,1,53,5
4343,2,19012016,2255.0,NaN,823.0,231080.0,28091927.0,488,1,1.0,...,*A419,*N390,*N19X,*N40X,*J449*J459,N40,88,1,53,5


A seguir exibem-se os histogramas do perfil de óbitos cuja caula base é Asma por idade nas regiões do Brasil:

In [134]:
onlyAsma['Norte']=onlyAsma[onlyAsma['CODREG']==1]['AGE']
onlyAsma['Nordeste']=onlyAsma[onlyAsma['CODREG']==2]['AGE']
onlyAsma['Sudeste']=onlyAsma[onlyAsma['CODREG']==3]['AGE'] 
onlyAsma['Sul']=onlyAsma[onlyAsma['CODREG']==4]['AGE'] 
onlyAsma['CentroOeste']=onlyAsma[onlyAsma['CODREG']==5]['AGE'] 

colorway = ['#A174C7', '#93D1D6', '#69CC69', '#FF4C5E', '#DBBD8A']

trace1 = go.Histogram(x=onlyAsma['Norte'],histnorm='percent', name='Norte',nbinsx=16,opacity=0.6)
trace2 = go.Histogram(x=onlyAsma['Nordeste'],histnorm='percent', name='Nordeste',nbinsx=16,opacity=0.6)
trace3 = go.Histogram(x=onlyAsma['Sudeste'],histnorm='percent', name='Sudeste',nbinsx=16,opacity=0.6)
trace4 = go.Histogram(x=onlyAsma['Sul'],histnorm='percent', name='Sul',nbinsx=16,opacity=0.6)
trace5 = go.Histogram(x=onlyAsma['CentroOeste'],histnorm='percent', name='CentroOeste',nbinsx=16,opacity=0.6)
data = [trace1, trace2, trace3, trace4, trace5]
layout = go.Layout(
    title='Histograma de Óbitos',
    xaxis=dict(
        title='Idade'
    ),
    yaxis=dict(
        title='Percentual do número de óbitos da região'
    ),
    bargap=0.1,
    barmode='overlay',colorway=colorway
)
fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename='histObAsma.html')

'histObAsma.html'

In [ ]:
# Contabilizacao de mortes de asma como causa primaria - regiao
obREGs = onlyAsma['CODREG'].value_counts()

# Contabilizacao de mortes com asma como uma das causas - regiao
obAlsoAsmaREGs = alsoAsma['CODREG'].value_counts()

# Contabilizacao de mortes por doenças respiratorias gerais - regiao
allDisease = allDisease['CODREG'].value_counts()

In [ ]:
obAlsoAsmaREGs

In [ ]:
propOB = []
for i in range (1,6):
    propOB.append(float(obREGs[i])/float(cidadesBR[cidadesBR['CODREG']==i].sum()['VAR01']))
propOB100 = np.multiply(propOB,100000)
propOB100 = np.append(propOB100,propOB100[0])
propOB100

In [ ]:
propOBAlsoAsma = []
for i in range (1,6):
    propOBAlsoAsma.append(float(obAlsoAsmaREGs[i])/float(cidadesBR[cidadesBR['CODREG']==i].sum()['VAR01']))
propOBAlsoAsma100 = np.multiply(propOBAlsoAsma,100000)
propOBAlsoAsma100 = np.append(propOBAlsoAsma100,propOBAlsoAsma100[0])
propOBAlsoAsma100

In [ ]:
regions = ['Norte','Nordeste','Sudeste','Sul','Centro-Oeste']


In [ ]:
N = len(regions)
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]
angles

In [ ]:

# What will be the angle of each axis in the plot? (we divide the plot / number of variable)
# Initialise the spider plot
ax = plt.subplot(111, polar=True)
# Draw one axe per variable + add labels labels yet
plt.xticks(angles[:-1], categories, color='grey', size=8)
# Draw ylabels
ax.set_rlabel_position(0)
plt.yticks([0,1.5,3], ["0","1,5","3,0"], color="grey", size=7)
plt.ylim(0,5)

# plot 01 - asma como causa primaria
ax.plot(angles,propOB100 , linewidth=1, linestyle='solid',label="Asma causa primaria")
ax.fill(angles, propOB100, 'b', alpha=0.2)


# plot 01 - asma como causa listada
ax.plot(angles,propOBAlsoAsma100 , linewidth=1, linestyle='solid',label="Asma causa nao primaria")
ax.fill(angles, propOBAlsoAsma100, 'r', alpha=0.2)
plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))

plt.savefig('spider01.png',dpi=192)